In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
import wandb
from transformers import AutoImageProcessor, ResNetForImageClassification

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Setup wandb configuration
def setup_wandb():
    wandb.init(project="computer_vision", config={
        "learning_rate": 0.001,
        "architecture": "ResNet50",
        "dataset": "alzheimer",
        "epochs": 2000,
        "batch_size": 64,
        "input_shape": (176, 208, 3),
        "num_classes": 4
    })
    return wandb.config

config = setup_wandb()



train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.8,1.2],
    zoom_range=0.2,
    shear_range=0.4

)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'alzheimer/train',
    target_size=config.input_shape[:2],
    batch_size=config.batch_size,
    class_mode='categorical'
)
test_generator = test_datagen.flow_from_directory(
    'alzheimer/test',
    target_size=config.input_shape[:2],
    batch_size=config.batch_size,
    class_mode='categorical'
)

def create_resnet_model(config):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=config.input_shape)
    for layer in base_model.layers[:-10]:
        layer.trainable = False
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)  # Add dropout
    x = layers.Dense(128, activation='relu')(x)
    predictions = layers.Dense(config.num_classes, activation='softmax')(x)
    return Model(inputs=base_model.input, outputs=predictions)

model = create_resnet_model(config)



# Customizing the Adam optimizer with specific beta_1, beta_2, and epsilon values
optimizer = Adam(
    learning_rate=config.learning_rate,  # Ensure 'config.learning_rate' is defined previously
    beta_1=0.8737880187038923,    # Equivalent to the first element of 'betas' in PyTorch
    beta_2=0.999,  # Equivalent to the second element of 'betas' in PyTorch
    epsilon=0.00000006513856488696   # Smoothing term to avoid division by zero, similar to PyTorch's default
)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


model_dir = 'model'
os.makedirs(model_dir, exist_ok=True)

checkpoint_callback = ModelCheckpoint(
    os.path.join(model_dir, 'model_{epoch:02d}_{val_accuracy:.4f}.keras'),
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)

class CustomWandbCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        wandb.log(logs)

class_weights = {
    0: 1.2,   # Non Demented
    1: 1.2,   # Very Mild Demented
    2: 1.7,   # Mild Demented
    3: 2    # Moderate Demented
}
history = model.fit(
    train_generator,
    epochs=config.epochs,
    class_weight=class_weights,
    validation_data=test_generator,
    callbacks=[checkpoint_callback, CustomWandbCallback()]
)

test_loss, test_acc = model.evaluate(test_generator)
wandb.log({'test_loss': test_loss, 'test_accuracy': test_acc})
print('Test accuracy:', test_acc)

model_filename = f'alzheimer_model_resnet50_{test_acc:.4f}.keras'
model.save(os.path.join(model_dir, model_filename))
print(f"Model saved successfully as '{model_filename}'.")